In [2]:
conda install pandas tensorflow

Channels:
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/yattmeo/Documents/GitHub/DMBAT7/.conda

  added / updated specs:
    - pandas
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20220623.0      |       hddbf539_6          16 KB  conda-forge
    cryptography-42.0.5        |   py39h30e54ef_0         1.2 MB
    flatbuffers-22.12.06       |       hf0c8a7f_2         1.2 MB  conda-forge
    google-auth-oauthlib-0.4.4 |     pyhd3eb1b0_0          18 KB
    grpc-cpp-1.51.1            |       h88f4db0_1          21 KB  conda-forge
    grpcio-1.51.1              |   py39h1890eb7_1         666 KB  conda-forge
    icu-70.1                   |       h96cf925_0        13.1 MB  conda-forge
    keras-2.11.0               |           py39_0         1.6 MB
    libabseil-20220623.0       | cxx17_h84

In [3]:
conda install numpy matplotlib scikit-learn

Channels:
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/yattmeo/Documents/GitHub/DMBAT7/.conda

  added / updated specs:
    - matplotlib
    - numpy
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mkl_fft-1.3.1              |   py39h7ae3660_1         186 KB  conda-forge
    mkl_random-1.2.2           |   py39h4d6be9b_0         317 KB  conda-forge
    scipy-1.12.0               |   py39ha516a68_0        21.9 MB
    ------------------------------------------------------------
                                           Total:        22.4 MB

The following NEW packages will be INSTALLED:

  libopenblas        pkgs/main/osx-64::libopenblas-0.3.21-h54e7dc3_0 

The following packages will be UPDATED:

  blas                                              1.0-mkl --> 1.0-openblas 
  mkl_fft           

In [4]:
conda install tensorflow --upgrade

usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: unrecognized arguments: --upgrade

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('Dataset/feature_output_10k_final.csv')
df = df.sample(n=5000, random_state=28)

# Assuming all other columns except 'label' are features
# Define a mapping dictionary
mapping = {'Human': 0, 'AI': 1}

# Create a new column 'label_numeric' with mapped values
df['label_numeric'] = df['label'].map(mapping)
y = df['label_numeric']

non_numeric_columns = df.select_dtypes(exclude=['number']).columns
X = df.drop(non_numeric_columns, axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

# Manually normalize the feature data
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std  # Use the mean and std from training data for test data

# If you have NaNs in your dataset, you may want to handle them before normalization
# For example, replacing NaNs with zeros (assuming NaNs represent missing values)
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print(X)

print("Training set class distribution:")
print(y_train.value_counts())

print("\nTesting set class distribution:")
print(y_test.value_counts())

# Verify that the mean and standard deviation of the testing set features match those of the training set
print("Mean of X_test features after normalization:", X_test.mean(axis=0))
print("Standard deviation of X_test features after normalization:", X_test.std(axis=0))

      Unnamed: 0.1  Unnamed: 0  generated  length    neg    neu    pos  \
4345          4345      473357        0.0    2438  0.015  0.842  0.142   
5182          5182       43015        1.0    2432  0.069  0.790  0.141   
1597          1597      445674        0.0    1120  0.042  0.867  0.091   
4459          4459      388152        1.0    2550  0.041  0.780  0.178   
4168          4168      383814        0.0    2021  0.048  0.812  0.139   
...            ...         ...        ...     ...    ...    ...    ...   
3616          3616      146847        1.0    1316  0.047  0.628  0.325   
7903          7903      238072        0.0    2656  0.032  0.879  0.089   
3917          3917      212425        0.0    2643  0.094  0.709  0.198   
2468          2468      245377        0.0    4209  0.050  0.738  0.212   
6713          6713      378397        0.0    5570  0.031  0.872  0.098   

      compound  subjectivity_score  flesch_reading_ease_score  \
4345    0.9946            0.537619            

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

#Define the model
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

#Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

2024-03-26 12:08:47.160572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 12:09:56.163731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
100/100 [==============================] - 1s 2ms/step - loss: 0.2456 - accuracy: 0.9137 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 2/10
100/100 [==============================] - 0s 823us/step - loss: 0.0202 - accuracy: 0.9991 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 3/10
100/100 [==============================] - 0s 791us/step - loss: 0.0084 - accuracy: 0.9991 - val_loss: 4.6299e-04 - val_accuracy: 1.0000
Epoch 4/10
100/100 [==============================] - 0s 848us/step - loss: 0.0035 - accuracy: 0.9997 - val_loss: 1.8133e-04 - val_accuracy: 1.0000
Epoch 5/10
100/100 [==============================] - 0s 778us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 9.8135e-05 - val_accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 0s 834us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 5.3938e-05 - val_accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 0s 839us/step - loss: 9.3455e-04 - accuracy: 1.0000 - val_loss: 3.58

In [8]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

32/32 [==============================] - 0s 530us/step - loss: 8.8683e-06 - accuracy: 1.0000

Test accuracy: 1.0


In [9]:
#This neural network is only relying on the paragraphs of text to determine if Human or AI.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load the data from the CSV file
data = pd.read_csv('Dataset/feature_output_10k_final.csv')

# Split the data into features (X) and labels (y)
# Define a mapping dictionary
mapping = {'Human': 0, 'AI': 1}

# Create a new column 'label_numeric' with mapped values
df['label_numeric'] = df['label'].map(mapping)
y = df['label_numeric']

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X = df['text']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

# Pad sequences to ensure uniform length
max_length = 200  # Adjust this value based on the maximum length of your paragraphs
padded_sequences = pad_sequences(sequences, maxlen=max_length)

print("First 10 sequences:")
for sequence in sequences[:10]:
    print(sequence)

First 10 sequences:
[577, 233, 2953, 577, 233, 4556, 1234, 2481, 105, 26, 361, 17, 4, 495, 64, 4, 89, 311, 45, 12, 64, 4, 272, 12, 98, 2, 64, 1, 99, 22, 457, 390, 59, 105, 28, 12, 101, 302, 55, 131, 12, 44, 302, 4, 192, 5, 22, 31, 9, 15, 53, 12, 64, 1, 173, 311, 390, 3496, 15, 53, 89, 1, 1613, 5, 180, 26, 22, 457, 3, 9, 487, 12, 4, 89, 266, 5, 318, 45, 180, 4, 272, 12, 98, 2, 64, 1, 173, 311, 7, 8, 56, 721, 180, 4, 272, 28, 267, 2, 35, 33, 9, 8, 264, 2, 64, 1, 173, 311, 45, 101, 936, 55, 131, 2, 423, 2, 4458, 12, 423, 2, 55, 131, 104, 12, 101, 64, 1, 311, 17, 56, 55, 131, 713, 3, 225, 516, 7, 1, 311, 8, 3907, 70, 20, 38, 1, 272, 7, 62, 197, 59, 45, 180, 4, 272, 302, 26, 82, 55, 131, 6, 433, 302, 4, 192, 5, 22, 59, 12, 15, 64, 4, 272, 7, 1, 426, 25, 75, 49, 1, 22, 896, 32, 38, 472, 26, 22, 56, 2, 35, 32, 64, 59, 66, 26, 22, 457, 45, 12, 423, 2, 65, 55, 25, 298, 22, 13, 369, 198, 12, 41, 1526, 476, 5, 56, 68, 22, 14, 121, 2, 98, 225, 516, 380, 98, 65, 55, 25, 298, 22, 1571, 21, 5473, 13,

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# Define the maximum length of the input sequences (i.e., the maximum number of words in each paragraph)
max_length = padded_sequences.shape[1]

# Define the size of the vocabulary (plus 1 to account for the padding token)
vocab_size = len(word_index) + 1

# Define the neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100))
model.add(LSTM(units=128))
model.add(Dense(units=1, activation='sigmoid'))

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
125/125 [==============================] - 14s 104ms/step - loss: 0.3664 - accuracy: 0.8290 - val_loss: 0.1566 - val_accuracy: 0.9480
Epoch 2/10
125/125 [==============================] - 12s 97ms/step - loss: 0.0767 - accuracy: 0.9778 - val_loss: 0.1236 - val_accuracy: 0.9660
Epoch 3/10
125/125 [==============================] - 12s 94ms/step - loss: 0.0332 - accuracy: 0.9918 - val_loss: 0.1738 - val_accuracy: 0.9440
Epoch 4/10
125/125 [==============================] - 13s 101ms/step - loss: 0.0175 - accuracy: 0.9960 - val_loss: 0.2383 - val_accuracy: 0.9470
Epoch 5/10
125/125 [==============================] - 12s 94ms/step - loss: 0.0627 - accuracy: 0.9803 - val_loss: 0.2636 - val_accuracy: 0.9460
Epoch 6/10
125/125 [==============================] - 13s 103ms/step - loss: 0.0399 - accuracy: 0.9885 - val_loss: 0.2098 - val_accuracy: 0.9390
Epoch 7/10
125/125 [==============================] - 12s 99ms/step - loss: 0.0179 - accuracy: 0.9948 - val_loss: 0.1741 - val_accura

In [13]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

32/32 [==============================] - 1s 32ms/step - loss: 0.2078 - accuracy: 0.9400

Test accuracy: 0.9399999976158142
